# 1. Mô hình embedding

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_Model = HuggingFaceEmbeddings(
    model_name="huyydangg/DEk21_hcmute_embedding",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\hdang\AppData\Local\Temp\ipykernel_1392\1278963481.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_Model = HuggingFaceEmbeddings(


# 2. Kết nối đến Qdrant

In [3]:
from langchain_qdrant import QdrantVectorStore

In [ ]:
exist_GD_Collection = QdrantVectorStore.from_existing_collection(
    embedding = embeddings_Model,
    prefer_grpc=True,
    url = "http://localhost:6333/",
    collection_name = "Domain_Law",
	metadata_payload_key="metadata"
)

# 3. Search không kèm Filter

In [ ]:
user_query = "người bao nhiêu tuổi thì mới được phép tham gia lao động ?"

In [12]:
# Thực hiện tìm kiếm trên Qdrant với filter `should`
search_results = exist_GD_Collection.similarity_search_with_score(
    query=user_query,
    filter=None,
    k=5
)

In [13]:
def print_Results_MoreKeywords(results):
    for result in results:
        doc = result[0]  # lấy phần tử đầu tiên
        score_Qdrant = result[1]  # lấy phần tử thứ hai
        print("Page_Content:\n", doc.page_content)
        print("Stt:",doc.metadata['STT'])
        print("Điều:",doc.metadata["Article"])
        print("Score Qdrant:",score_Qdrant,"\n","-----------------------")

In [14]:
print_Results_MoreKeywords(search_results)

Page_Content:
 1 . Lao_động chưa thành_niên là người lao_động chưa đủ 18 tuổi . 2 . Người từ đủ 15 tuổi đến chưa đủ 18 tuổi không được làm công_việc hoặc làm_việc ở nơi làm_việc quy_định tại Điều 147 của Bộ_luật này . 3 . Người từ đủ 13 tuổi đến chưa đủ 15 tuổi chỉ được làm công_việc nhẹ theo danh_mục do Bộ_trưởng Bộ Lao_động - Thương_binh và Xã_hội ban_hành . 4 . Người chưa đủ 13 tuổi chỉ được làm các công_việc theo quy_định tại khoản 3 Điều 145 của Bộ_luật này .
Stt: 13
Điều: Điều 143. Lao động chưa thành niên
Score Qdrant: 0.6392779350280762 
 -----------------------
Page_Content:
 4 . Người học nghề , người tập nghề phải đủ 14 tuổi trở lên và phải có đủ sức_khỏe phù_hợp với yêu_cầu học nghề , tập nghề . Người học nghề , người tập nghề thuộc danh_mục nghề , công_việc nặng_nhọc , độc_hại , nguy_hiểm hoặc đặc_biệt nặng_nhọc , độc_hại , nguy_hiểm do Bộ_trưởng Bộ Lao_động - Thương_binh và Xã_hội ban_hành phải từ đủ 18 tuổi trở lên , trừ lĩnh_vực nghệ_thuật , thể_dục , thể_thao . 5 . Tro